# Exercises on thermal policies
Combine both gem5-X, power model, and 3D-ICE

## Simple thermal policy with frequency scaling

1. Run 3D-ICE with given power trace @ x GHz
2. Plot the temperature trace
3. Check if the temperature is within the safe range
4. Lower down the frequency to x GHz and run 3D-ICE again
5. Plot the temperature trace
6. Check if the temperature is within the safe range

## Thermal policy for 3D MPSoC


1. Enable all of the cores and run 3D-ICE with given power trace @ x GHz
2. Plot the temperature trace
3. Check if the temperature is within the safe range
4. Disable some of the cores and run 3D-ICE again
5. Plot the temperature trace
6. Check if the temperature is within the safe range